In [ ]:
#[[Colab]](https://colab.research.google.com/github/JDekun/DCNet/blob/dcnet/fcn/scripts/debug.ipynb?authuser=1) 
!git clone https://github.com/JDekun/Kaggle.git
github = "https://github.com/JDekun/DCNet"  #@param {type:"string"}
branch = "dcnet"  #@param {type:"string"}
work_path = "/fcn/scripts" #@param {type:"string"}
datasets = "huanghanchina/pascal-voc-2012,mydekun/pre-trained" #@param {type:"string"}
!python Kaggle/colab_env.py --github $github --branch $branch --datasets $datasets
import os; work_path = "/content/working/" + github.rsplit('/',1)[-1].rsplit('.',1)[0] + work_path
os.chdir(work_path)

In [ ]:
pip -q install wandb > /dev/null 2>&1

In [ ]:
!experiment_name="debug";cd ../ ;\
    name_date="${experiment_name}/$(date +%Y-%m%d-%H%M%S)";\
    dir="./results/${name_date}";mkdir -p ${dir};dir_log="${dir}/output.log";\
CUDA_VISIBLE_DEVICES=0 torchrun --nproc_per_node=1 train_multi_GPU.py \
    --wandb False --wandb_model dryrun --sync_bn False --amp True --aux False \
    --batch_size 20 --batch_size_val 8 \
    --epochs 40 --start_epoch 0 --data_path pascal-voc-2012 \
    --model_name dcnet_resnet50 --pre_trained pre-trained/pytorch/fcn_resnet50_coco.pth \
    --contrast -1 --loss_name double --L1_loss 0.1 --L2_loss 0.1 --L3_loss 0.1\
    --name_date $name_date \
    2>&1 | tee $dir_log

In [ ]:
!experiment_name="new";cd ../ ;\
    name_date="${experiment_name}/$(date +%Y-%m%d-%H%M%S)";\
    dir="./results/${name_date}";mkdir -p ${dir};dir_log="${dir}/output.log";\
CUDA_VISIBLE_DEVICES=1,2,3 torchrun --nproc_per_node=3 train_multi_GPU.py \
    --wandb True --wandb_model dryrun --sync_bn False --amp True --aux False \
    --batch_size 10 --batch_size_val 8 \
    --epochs 40 --start_epoch 0 \
    --model_name dcnet_resnet50 --pre_trained fcn_resnet50_coco \
    --contrast 0 --loss_name double --L1_loss 0.1 --L2_loss 0.1 --L3_loss 0.1\
    --name_date $name_date \
    2>&1 | tee $dir_log

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 1): env://
| distributed init (rank 2): env://
| distributed init (rank 0): env://
new/2022-1204-145334
Namespace(L1_loss=0.1, L2_loss=0.1, L3_loss=0.1, amp=True, aux=False, batch_size=10, batch_size_val=8, checkpoint_dir='./results/new/2022-1204-145334', contrast=0, data_path='../../../input/pascal', device='cuda', dist_backend='nccl', dist_url='env://', distributed=True, epochs=40, gpu=0, loss_name='double', lr=0.0001, model_name='dcnet_resnet50', momentum=0.9, name_date='new/2022-1204-145334', num_classes=20, pre_trained='fcn_resnet50_coco', print_freq=5, project_dim=128, rank=0, resume='', seed=304, start_epoch=0, sync_bn=False, test_only=False, wandb=True, wandb_model